In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
### change the url
# http://video.google.com/timedtext?lang=en&v=

url_list = pd.read_csv('/Users/Amiros/GitHub/100_days_of_spice/transcripts/url_list.csv')

In [3]:
url_list

,date,link
0,1_23,http://video.google.com/timedtext?lang=en&v=hT...
1,1_24,http://video.google.com/timedtext?lang=en&v=lV...
2,1_25,http://video.google.com/timedtext?lang=en&v=Oa...
3,1_30,http://video.google.com/timedtext?lang=en&v=b9...
4,1_31,http://video.google.com/timedtext?lang=en&v=CM...
5,2_1,http://video.google.com/timedtext?lang=en&v=F8...
6,2_2,http://video.google.com/timedtext?lang=en&v=bq...
7,2_3,http://video.google.com/timedtext?lang=en&v=BW...
8,2_7,http://video.google.com/timedtext?lang=en&v=f1...
9,2_8,http://video.google.com/timedtext?lang=en&v=XO...


In [4]:
for i in range(url_list.shape[0]):
    try:
        url = requests.get(url_list['link'][i])
        soup = BeautifulSoup(url.text, 'html.parser')
        content = soup.findAll('text')
        
        # creating a dataframe from the content
        lines = []
        df = []
        for line in content:
            lines.append(str(line))
        df = pd.DataFrame(lines)
        
        # splitting rows
        df = pd.DataFrame(df[0].str.split('">',1).tolist(),
                                           columns = ['time','text'])
        s = df['time'].apply(lambda x: x.split(' '))
        df['start'] = s.apply(lambda x: x[-1])
        df['duration'] = s.apply(lambda x: x[1])
        df = df.drop(['time'], axis=1)
        
        # trimming df
        df['duration'] = df['duration'].str.replace('dur="','')
        df['start'] = df['start'].str.replace('start="','')
        df['text'] = df['text'].str.replace('</text>','')
        df['duration']= df['duration'].str.replace('"','')
        df['text'] = df['text'].str.replace('\n',' ')
        
        filename = ("transcripts/" + url_list['date'][i] + "-transcript.csv")
        df.to_csv(filename, sep=',', index=False)
    except:
        pass      